# 確率的勾配降下法
$W \gets nW$


In [ ]:
# 確率的勾配降下法 (いままで使ってたやり方)
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.key():
            params[key] -= self.lr * grads[key]
            


# Momentum法

$v \gets av - nW$

勾配が０に使づいてほとんど動かなくなってもaによって動く(-nWと逆の符号なのでブレーキ的な働きをする)

In [ ]:
import numpy as np
# Momentum法
class Momentum:
    def __init__(self, lr=0.01, momentum):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zero_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

    












# AdamGrad
$$
   g_t = \nabla f(\theta_{t-1})
$$

$$
   m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t
$$
$$
v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t \odot g_t
$$
$$
   \hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
$$
$$
   \theta_t = \theta_{t-1} - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}
$$

前回の勾配のノルムが大きいほど更新時の値は小さくなる(vtのおかげで)


In [ ]:
import numpy as np
# AdamGrad
class AdamGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zero_like(val)
                
        for key in params.keys():
            # アダマール積
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)


# 初期値の選定方法
色々らしい。
- 適当に０に近い値とか選ぶとどの層の分布も同じような値になったり、極端な値に偏ったりする。
- 適切な初期値を与えるのは難しい問題らしい。
- xavierの初期値という方法がある。

## chatgptによるxavierの初期値の説明
### Xavier Initialization の数理的背景

Xavier 初期化は、ニューラルネットワークの学習を安定させるための重みの初期化手法です。この手法では、各層において重みが平均 0、特定の標準偏差を持つ正規分布または一様分布からサンプリングされます。

---

### 基本アイデア

Xavier 初期化は、以下の2つを目的としています：
1. **フォワードパスの出力が適切にスケールされること**：
   - 各層での出力の分散が極端に大きくなったり小さくなったりしない。
2. **バックプロパゲーションの勾配が適切にスケールされること**：
   - 勾配爆発や勾配消失を防ぎ、学習を安定させる。

---

### 数式

重み行列 \(W\) を次のように初期化します：

### 1. 正規分布の場合
\[
W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{n_{\text{in}} + n_{\text{out}}}}\right)
\]

- 平均：\(0\)
- 標準偏差：\(\sqrt{\frac{2}{n_{\text{in}} + n_{\text{out}}}}\)

### 2. 一様分布の場合
\[
W \sim U\left(-\sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}, \sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}\right)
\]

- 範囲：\(\pm \sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}\)

ここで：
- \(n_{\text{in}}\)：現在の層への入力ユニット数
- \(n_{\text{out}}\)：現在の層からの出力ユニット数

---

### 例：入力層から隠れ層への初期化

仮に以下の設定があるとします：
- 入力ユニット数：\(n_{\text{in}} = 784\)（MNISTの画像ピクセル数）
- 出力ユニット数：\(n_{\text{out}} = 128\)（隠れ層のニューロン数）

### 正規分布の場合
標準偏差は次のように計算されます：
\[
\sigma = \sqrt{\frac{2}{n_{\text{in}} + n_{\text{out}}}} = \sqrt{\frac{2}{784 + 128}} = \sqrt{\frac{2}{912}} \approx 0.0469
\]

重みは次の分布に従います：
\[
W \sim \mathcal{N}\left(0, 0.0469\right)
\]

### 一様分布の場合
範囲は次のように計算されます：
\
\pm \sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}} = \pm \sqrt{\frac{6}{784 + 128}} = \pm \sqrt{\frac{6}{912}} \approx \pm 0.0814
\]

重みは次の分布に従います：
\[
W \sim U\left(-0.0814, 0.0814\right)
\]

---

### Xavier 初期化の重要性

1. **バランスの取れたスケール**：
   - 各層での出力や勾配のスケールが均一に保たれます。
2. **勾配消失・爆発の防止**：
   - 重みの初期化が適切な範囲で行われるため、学習が安定します。
3. **活性化関数との相性**：
   - 活性化関数が非線形性を発揮しやすくなり、学習効率が向上します。

---

### 参考

- Xavier 初期化は、活性化関数としてシグモイドや双曲線正接（tanh）を使用する場合に特に有効です。
- ReLU の場合には、**He 初期化**がより適切な場合があります（\(\sqrt{\frac{2}{n_{\text{in}}}}\) を使用）。

---


chatgptによるxavierの初期値の説明ここまで

これの何がうれしいかというと、入力パラメータ数に関係なく正規分布でばらつくので偏りがなくなる。